In [ ]:
import logging
import select
import socket


class EthComm(object):
    def __init__(self, host, port, EOL='\r\n'):
        object.__init__(self)
        self.sock = None
        self.host = host
        self.port = port
        self.EOL = EOL

    def connectSock(self, timeout=10.):
        """| Connect socket if self.sock is None.

        :return: - socket
        """
        if self.sock is None:
            s = self.createSock()
            s.settimeout(timeout)
            s.connect((self.host, self.port))

            self.sock = s

        return self.sock

    def createSock(self):
        return socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    def closeSock(self):
        """| Close the socket.

        :raise: Exception if closing socket has failed
        """
        try:
            self.sock.close()
        except:
            pass

        self.sock = None

    def sendOneCommand(self, cmdStr, doClose=False, cmd=None):
        """| Send one command and return one response.

        :param cmdStr: (str) The command to send.
        :param doClose: If True (the default), the device socket is closed before returning.
        :param cmd: on going command
        :return: reply : the single response string, with EOLs stripped.
        :raise: IOError : from any communication errors.
        """

        if cmd is None:
            cmd = self.actor.bcast

        fullCmd = ('%s%s' % (cmdStr, self.EOL)).encode('utf-8')
        print('sending %r', fullCmd)

        s = self.connectSock()

        try:
            s.sendall(fullCmd)

        except:
            self.closeSock()
            raise

        reply = self.getOneResponse(sock=s, cmd=cmd)

        if doClose:
            self.closeSock()

        return reply

    def getOneResponse(self, sock=None, cmd=None):
        """| Attempt to receive data from the socket.

        :param sock: socket
        :param cmd: command
        :return: reply : the single response string, with EOLs stripped.
        :raise: IOError : from any communication errors.
        """
        if sock is None:
            sock = self.connectSock()

        ret = self.ioBuffer.getOneResponse(sock=sock, cmd=cmd)
        reply = ret.strip()

        print('received %r', reply)

        return reply


class BufferedSocket(object):
    """ Buffer the input from a socket and block it into lines. """

    def __init__(self, name, sock=None, loggerName=None, EOL='\n', timeout=10.,
                 logLevel=logging.INFO):
        self.EOL = EOL
        self.sock = sock
        self.name = name
        self.timeout = timeout

        self.buffer = ''

    def getOutput(self, sock=None, timeout=None, cmd=None):
        """ Block/timeout for input, then return all (<=1kB) available input. """

        if sock is None:
            sock = self.sock
        if timeout is None:
            timeout = self.timeout

        readers, writers, broken = select.select([sock.fileno()], [], [], timeout)
        if len(readers) == 0:
            cmd.warn('text="Timed out reading character from %s"' % self.name)
            raise IOError

        return sock.recv(1024).decode('utf8', 'ignore')

    def getOneResponse(self, sock=None, timeout=None, cmd=None, doRaise=False):
        """ Return the next available complete line. Fetch new input if necessary.

        Args
        ----
        sock : socket
           Uses self.sock if not set.
        timeout : float
           Uses self.timeout if not set.

        Returns
        -------
        str or None : a single line of response text, with EOL character(s) stripped.
        """
        while self.buffer.find(self.EOL) == -1:
            try:
                more = self.getOutput(sock=sock, timeout=timeout, cmd=cmd)
                if not more:
                    if doRaise:
                        raise IOError
                    else:
                        return self.getOneResponse(sock=sock, timeout=timeout, cmd=cmd, doRaise=True)

            except IOError:
                return ''

            print('%s added: %r' % (self.name, more))
            self.buffer += more

        eolAt = self.buffer.find(self.EOL)
        ret = self.buffer[:eolAt]

        self.buffer = self.buffer[eolAt + len(self.EOL):]

        return ret

class Cmd(object):
    def __init__(self):
        self.warn = print
        self.inform = print
        self.fail = print
        self.finish = print
    

In [ ]:
host = 'filterwheel-dcb'
port = 9000
EOL = '\r\n'

In [ ]:
cmd = Cmd()

In [ ]:
sock = EthComm(host=host, port=port, EOL='\r\n')
sock.ioBuffer = BufferedSocket('socketIO', EOL='\n', timeout=5)
s = sock.connectSock()

In [ ]:
wheel = 'linewheel'
position = '1'

In [ ]:
ret = sock.sendOneCommand(f'{wheel} {position}', cmd=cmd)
cmd.inform(f'text="{ret}"')

while 'Moved to position' not in ret:
    ret = sock.getOneResponse(cmd=cmd)
    cmd.inform(f'text="{ret}"')

__, position = ret.split('Moved to position')
position = int(position)

In [ ]:
ret = sock.sendOneCommand(f'adccalib ', cmd=cmd)

In [ ]:
ret = sock.sendOneCommand(f'continue ', cmd=cmd)

In [ ]:
ret[-1]

In [ ]:
cmdStr = 'adccalib '
EOL = '\r\n'

In [ ]:
fullCmd = ('%s%s' % (cmdStr, EOL)).encode('latin-1')
s = sock.connectSock()

In [ ]:
s.sendall(fullCmd)

In [ ]:
s.recv(1024)

In [ ]:
sock.closeSock()

In [ ]:
s.sendall('toto'.encode('latin-1'))

In [ ]:
print('port0 = |UL|  port1 = |LL| \nCalibrating FW 1 \nattached 2 filter wheel(s): \nindex 0: ID 0 Name EFW \nindex 1: ID 1 Name EFW \nselecting 1\nCalibrating \nDone\n')

In [ ]:
s.close()

In [ ]:
print(s)

In [ ]:
ret.split('\r')